<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Generate Readme for Awesome Notebooks

## Input

### Import librairies

In [ ]:
import os
import requests
import pandas as pd
import naas_drivers
import urllib.parse
import json
import copy
import markdown
import nbformat
from nbconvert import MarkdownExporter
from papermill.iorw import (
    load_notebook_node,
    write_ipynb,
)
try:
    from git import Repo
except:
    !pip install GitPython
    from git import Repo

### Variables

In [ ]:
# README variables
readme_template = "README_template.md"
readme = "README.md"
replace_var = "[[DYNAMIC_LIST]]"

# Json output
json_file = "templates.json"

# Others
current_file = '.'
notebook_ext = '.ipynb'
github_url = 'https://github.com/jupyter-naas/awesome-notebooks/tree/master'
github_download_url = 'https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/'
naas_download_url ='https://app.naas.ai/user-redirect/naas/downloader?url='
naas_logo ='https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo=data:image/svg+xml;base64,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iVVRGLTgiPz4KPHN2ZyB3aWR0aD0iMTAyNHB4IiBoZWlnaHQ9IjEwMjRweCIgdmlld0JveD0iMCAwIDEwMjQgMTAyNCIgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIiB4bWxuczp4bGluaz0iaHR0cDovL3d3dy53My5vcmcvMTk5OS94bGluayIgdmVyc2lvbj0iMS4xIj4KIDwhLS0gR2VuZXJhdGVkIGJ5IFBpeGVsbWF0b3IgUHJvIDIuMC41IC0tPgogPGRlZnM+CiAgPHRleHQgaWQ9InN0cmluZyIgdHJhbnNmb3JtPSJtYXRyaXgoMS4wIDAuMCAwLjAgMS4wIDIyOC4wIDU0LjUpIiBmb250LWZhbWlseT0iQ29tZm9ydGFhLVJlZ3VsYXIsIENvbWZvcnRhYSIgZm9udC1zaXplPSI4MDAiIHRleHQtZGVjb3JhdGlvbj0ibm9uZSIgZmlsbD0iI2ZmZmZmZiIgeD0iMS4xOTk5OTk5OTk5OTk5ODg2IiB5PSI3MDUuMCI+bjwvdGV4dD4KIDwvZGVmcz4KIDx1c2UgaWQ9Im4iIHhsaW5rOmhyZWY9IiNzdHJpbmciLz4KPC9zdmc+Cg=='

### Get files list

In [ ]:
repo = Repo('.')
branch = repo.active_branch
list_of_dir = f"https://api.github.com/repos/jupyter-naas/awesome-notebooks/git/trees/{branch.name}?recursive=1"
r_gh = requests.get(list_of_dir).json().get("tree")
notebooks = []
nb_to_skip = 0

for file in r_gh:
    if ".github" not in file.get("path") and ".gitignore" not in file.get("path"):
        if file.get("path").endswith(".ipynb"):
            temp = file.get("path").split("/")
            if temp == -1:
                data = {
                    "root": None,
                    "subdir": file.get("path")
                }
                notebooks.append(data)
            else:
                last_folder = ""
                file_name = temp[-1]
                temp.pop()
                for folder in temp:
                    last_folder += "/" + folder
                root = last_folder[1:]
                if root == "":
                    nb_to_skip += 1
                data = {
                    "root": root,
                    "subdir": file_name
                }
                notebooks.append(data)

df_github = pd.DataFrame(notebooks)
df_github

## Model

### Reformat functions

In [ ]:
def reformat_file_name(file):
    file_nice = file.replace('_', ' ')
    file_nice = file_nice.replace(notebook_ext, '')
    file_nice = file_nice.replace(folder_nice, '')
    file_nice = file_nice.strip()
    if (file_nice != ""):
            file_nice = file_nice[0].capitalize() + file_nice[1:]
    return file_nice

### Get functions

In [ ]:
def get_open_button(download_link):
    return f"""<a href="{download_link}" target="_parent"><img src="{naas_logo}"/></a>"""

def get_title(folder_nice, file_nice, download_link):
    return f"""# {folder_nice} - {file_nice}\n{get_open_button(download_link)}"""

def get_tags(text):
    result = []
    tags = text.split(' ')
    for tag in tags:
        if len(tag) >= 2 and tag[0] == '#' and tag[1] != ' ' and tag[1] != '#':
            result.append(tag)
    return result

### Set 'Naas Download' link on notebook

In [ ]:
def set_notebook_title_and_get_tags(notebook_path, title_source, final_title, good_format):
    header_found = False
    tag_found = False
    tags = None
    count = 0
    nb = load_notebook_node(notebook_path)
    nb = copy.deepcopy(nb)
    # Parse the entire notebook
    for cell in nb.cells:
        source = cell.source
        if cell.cell_type == "code":
            nb.cells[count].outputs = []
        # Get the tags, because tags are always after the header cell
        if header_found and not tag_found:
            if cell.cell_type == "markdown":
                tags = get_tags(cell.source)
                print(tags)
            tag_found = True
        # Get the header cell
        if not header_found and cell.cell_type == "markdown" and len(source) > 2 and source[0] == '#' and source[1] == ' ':
            nb.cells[count].source = title_source
            header_found = True
        count += 1
    # Set the good title format in the notabook
    write_ipynb(nb, notebook_path)
    # Rename the notebook if the tool name is not the same
    if good_format == 1:
        os.rename(notebook_path, final_title)
    # Return tags
    return tags

### Convert filepath in Markdown text

In [ ]:
def get_file_md(folder_nice, folder_url, files, json_templates, title_sep="##", subtitle_sep="*"):
    good_format = 0
    final_title = ""
    md = ""
    folder_name = ""
    tool_name = ""
    tool_title = ""

    if (len(files) > 0):
        md += f"\n{title_sep} {folder_nice}\n"
        for file in files:
            if file.endswith(notebook_ext):
                good_format = 0
                file_url = urllib.parse.quote(file)
                folder_name = folder_nice
                temp = folder_name.split("_")
                tool_name = temp[0]
                file_nice = reformat_file_name(file)
                # Check if the tool name is the same as the tool name in the notebook name
                if tool_name != folder_name:
                    temp = file.split("_")
                    del temp[0]
                    tool_title = folder_name + "_"
                    for i in temp:
                        tool_title += i + "_"
                    final_title = folder_name + "/" + tool_title[:-1]
                    good_format = 1
                path = urllib.parse.unquote(f"{folder_url}/{file_url}")
                # Get the download URL
                dl_url = f"{naas_download_url}{github_download_url}{folder_url}/{file_url}"
                # Put the title to the format "TOOLS - NAME_OF_NOTEBOOK Open_In_Naas"
                title = get_title(folder_nice, file_nice, dl_url)
                # Set the good title format and get the tags from the notebooks of the folder
                tags = set_notebook_title_and_get_tags(path, title, final_title, good_format)
                # Get the name of the Notebook and the redirect to github link
                nb_redirect = f"[{file_nice}]({github_url}/{folder_url}/{file_url})"
                # Get the open in naas format
                open_button = get_open_button(dl_url)
                # For the actual file, put the nnotebook name and the gihub link for the return in markdown
                md += f"{subtitle_sep} {nb_redirect}\n"
                json_templates.append({
                    'tool': folder_nice,
                    'notebook': file_nice,
                    'tags': tags,
                    'update': '',
                    'action': open_button
                })
    return md

### Generate markdown for each notebooks

In [ ]:
generated_list = ""
json_templates = []
list_of_tools = []
index_max = len(notebooks)
index = 0

while index <= (index_max - nb_to_skip - 1):
    folder_nice = notebooks[index].get("root")
    if folder_nice not in list_of_tools and folder_nice != "":
        md_round = ""
        files = []
        list_of_tools.append(folder_nice)
        folder_url = urllib.parse.quote(folder_nice)
        while notebooks[index].get("root") == folder_nice:
            files.append(notebooks[index].get("subdir"))
            index += 1
        if ("/" not in folder_nice):
            md_round += get_file_md(folder_nice, folder_url, files, json_templates)
        else:
            folder_url = urllib.parse.quote(folder_nice)
            subfolder_nice = folder_nice.split('/')[1].replace('_', ' ').replace(folder_nice, '').strip()
            md_round += get_file_md(subfolder_nice, folder_url, files, json_templates, "\t###", "\t-")
        generated_list += md_round

## Output

### Preview the generated list

In [ ]:
naas_drivers.markdown.display(generated_list)

### Generate readme for github repository

In [ ]:
# Open README template
template = open(readme_template).read()

# Replace var to get list of templates in markdown format
template = template.replace(replace_var, generated_list)

# Save README
f  = open(readme, "w+")
f.write(template)
f.close()

### Generate json for naas manager

In [ ]:
with open(json_file, 'w') as f:
    json.dump(json_templates, f)